## Kosarak


20210607:
    line 20: Sk[row][col]+=1 but not element.count
    line 41: when head_item.keep==0, replace maxID


In [4]:
from Node import DS
from Tools import Func
from Tools import Config
import pandas as pd
import numpy as np
import os
import time
import operator
import sys

# ==========================UpdateSk==========================
def UpdateSk(element,Sk_head,Sk):
    e_max=Config.e_max
    col,row=Func.position(element)
        # col / row index of element
    #print("{} send to Sk[{}][{}]".format(element,row,col))
    # ==========================update sketch==========================
    Sk_head[row].count+=element.count
    Sk_head[row].distinct.add(element.ID)
    Sk[row][col]+=1
    Update_local_max(Sk_head[row],Sk[row],element,col)
    Update_emax(Sk_head,Sk,row)

# ==========================update local max==========================       
def Update_local_max(head_item,element_list,element,column):
    # local max need only 1 row
    #print("In Update_local_max:")
    width,depth=Config.width,Config.depth
    if head_item.maxID=='':
        head_item.maxID=element.ID
        head_item.keep=1
    elif head_item.maxID==element.ID:
        head_item.keep+=1
    else:
        # head_item.maxID != element.ID:
        local_max_col,local_max_row=Func.position(DS.Tail(head_item.maxID,0))
            # element_list[local_max_col]: lhh的估計值
        if local_max_col==column:
            # lhh collision with e
            head_item.keep-=1
            if head_item.keep==0:
                head_item.maxID=element.ID
                head_item.keep=1
        elif element_list[local_max_col]<element_list[column]:
            # not collision
            head_item.maxID=element.ID
            head_item.keep=1

'''
        if local_max_col==column:
            # lhh collision with e
            head_item.keep-=1
            if head_item.keep<0:
                head_item.maxID=element.ID
                head_item.keep=1
        elif element_list[local_max_col]<element_list[column]:
            # not collision
            head_item.maxID=element.ID
            head_item.keep=1

        if element_list[local_max_col]<element_list[column]:
            head_item.maxID=element.ID
'''            

# ==========================update e_max==========================
def Update_emax(head,sketch,sk_row):
    local_max_col,local_max_row=Func.position(DS.Tail(head[sk_row].maxID,0))
    if sketch[local_max_row][local_max_col]>Config.e_max.count:
        Config.e_max.ID=head[sk_row].maxID
        Config.e_max.count=sketch[local_max_row][local_max_col]

# ========================== BringBack=========================
def BringBack(e_min,head,sketch):
    # bring e_max back to Top
    # e_min=e_max, e_max=Null, delete e_max.count in Sketch, send e_min into Sketch
    temp=DS.Tail(e_min.ID,e_min.count)
    e_min.ID=Config.e_max.ID
    e_min.count=Config.e_max.count
    c,r=Func.position(Config.e_max)
    head[r].bringback+=1
    DeleteSk(e_max,head,sketch)
    UpdateSk(temp,head,sketch)

# ==========================DeleteSk=========================
def DeleteSk(element,head,sketch):
    # e_max in sketch: sketch[r][c]=0, total count-=sketch[row][col]
    #width,depth=get_width_depth()
    col,row=Func.position(element)
    head[row].count-=element.count
        # total_count-=element.count
    sketch[row][col]=0
    head[row].maxID=''
    # reset e_max
    element.ID=""
    element.count=0

    
# ==========================main=========================    

filename='kosarak.dat'
dataset='kosarak'
filepath=r"..\dataset\kosarak"
src_data=os.path.join(filepath,filename)

w=128
d=128
size=4096
Config.Set_default(w,d,size)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
e_max=Config.e_max
    # initialize e_max
    
Sk_head=[DS.Head(0) for j in range(Config.depth)]
Sketch=np.zeros((Config.depth,Config.width),dtype='int32')

Top=[]

#item_count=100
#income=0
start=time.time()
with open(src_data,'r') as file:
    while True:
        e=file.readline().strip('\n')
        if not e:
            print('EOF')
            break
        else:
            # item_count-=1
            #income+=1
            #print("read {}-th element:{}".format(income,e))
            item=DS.Tail(e,1)
            index=Func.find(item,Top)
            if index<0:
                if len(Top)<Config.size:
                    Top.append(item)
                    # 此處的count必等於1，不用sorting
                    index=len(Top)-1
                else:
                    UpdateSk(item,Sk_head,Sketch)
            else:
                Top[index].count+=1
                if index==0 or Top[index].count< Top[index-1].count:
                    pass
                else:
                    Top.sort(key=operator.attrgetter('count'),reverse=True)                
        if Config.e_max.count>Top[len(Top)-1].count:
            BringBack(Top[len(Top)-1],Sk_head,Sketch)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            #print('Top after BringBack: \n\t{}'.format(Top)) 

end=time.time()
print("Top-{},Sketch:{}*{}".format(Config.size,Config.depth,Config.width))
print("Execution time:{:8.3f} seconds.".format(end-start))
print("Total memory {} bytes=".format(sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*Config.depth),end='')
print("Top:{} bytes, Sketch:{} bytes, Sketch_head:{} bytes.".format(sys.getsizeof(Top),Sketch.nbytes,sys.getsizeof(Sk_head[0])*Config.depth))

#====================Top to csv=============================
my_csv_file=Func.Top_to_csv(Top,dataset)
    # return "My_OO_kosarak_Top_512_128_256.csv"

#====================result compare=============================
# precision
gr_dir=r"..\dataset\kosarak"
gr_file_name='kosarak_ground_truth.csv'
gr_path=os.path.join(gr_dir,gr_file_name)
    # gr_path="..\dataset\kosarak\kosarak_ground_truth.csv"
my_result_path=os.path.join(r'..\result',my_csv_file)
    # my_result_path="..\result\My_OO_kosarak_Top_xx_xx_xx.csv"

true_positive_set,precision=Func.Get_precision(gr_path,my_result_path)

# ====================ARE/AAE for top and all====================

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,2)
endx=time.time()
print("Query2:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,3)
endx=time.time()
print("Query3:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,4)
endx=time.time()
print("Query4:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))


'''

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,1)
endx=time.time()

print("Query1:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,3)
endx=time.time()
print("Query3:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,4)
endx=time.time()
print("Query4:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))



startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,2)
endx=time.time()
print("Query2:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))
'''

# ====================Cover result into a dataframe====================
result_para=(startx,endx,true_positive_set,precision,top_are,top_aae,all_are,all_aae,tp,fp)
df=Func.To_df(Top,Sketch,Sk_head,*result_para)
df

EOF
Top-4096,Sketch:128*128
Execution time: 464.202 seconds.
Total memory 106632 bytes=Top:33928 bytes, Sketch:65536 bytes, Sketch_head:7168 bytes.
My_OO_kosarak_Top_4096_128_128.csv
Precision:   0.4312
lhh_are:0.09401971726190476,lhh_aae:0.609375
Query2:
Find:1766, TP:1766, FP:2330
Top_ARE: 0.192146
Top_AAE: 3.057758
all_ARE: 0.100542
all_AAE: 0.212172
Estimate time: 241.256 seconds.
lhh_are:0.09401971726190476,lhh_aae:0.609375
Query3:
Find:1766, TP:1766, FP:2330
Top_ARE: 0.192146
Top_AAE: 3.057758
all_ARE: 0.100542
all_AAE: 0.212172
Estimate time: 229.136 seconds.
lhh_are:0.09401971726190476,lhh_aae:0.609375
Query4:
Find:1766, TP:1766, FP:2330
Top_ARE: 0.192146
Top_AAE: 3.057758
all_ARE: 0.100542
all_AAE: 0.212172
Estimate time: 251.929 seconds.


,Top-k,Sketch,Total memory,Exe_time,Find,Precision,ARE-Top,AAE-Top,ARE-all,AAE-all
0,4096,128*128,106632 bytes =Top:33928+ Sketch:65536+ Sk_head...,251.929,"Find:1766, TP:1766, FP:2330",0.4312,0.192146,3.057758,0.100542,0.212172


In [ ]:
# 各部份的summation

#print([i.count for i in Top])
gr_path=r"..\dataset\kosarak"
gr_file_name='kosarak_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(gr_path,gr_file_name))

print("Top[{}]:truth:{}, algo result:{}".format(Config.size,grtruth['Count'][:Config.size].sum(),sum([i.count for i in Top])))
print("sum of Sketch:{}".format(Sketch.sum()))
print("sum of Sk_head:{}".format(sum([j.count for j in Sk_head])))

# print("sum of all count of algo:{}".format(sum([i.count for i in Top])+Sketch.sum()))
print("sum of Top + Sk_head: {}".format(sum([i.count for i in Top])+sum([j.count for j in Sk_head])))
print("all arriving element: {}".format(grtruth['Count'].sum()))


In [ ]:
c,r=Func.position(DS.Tail('648 91',0))
ratio=Config.width/row_cardinality[r]
Sketch[r][c]
x=Sk_head[r].keep+int((Sketch[r][c]-Sk_head[r].keep)*ratio)
Sketch[r][c],x,ratio

In [ ]:
# 產生多個hash functions output
import mmh3
from Crypto.Util import number

inputs=['a','c','d']
seed = [number.getPrime(32) for i in range(len(inputs))]
for x,y in zip(inputs,seed):
    print(mmh3.hash(x,y,signed=False))
print(seed)